In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Mengambil data dari API
response = requests.get("http://localhost:3000/api/sales")
data = pd.read_json(response.text)

# Mengubah kolom 'date' menjadi tipe datetime
data['date'] = pd.to_datetime(data['date'])

# Filter data berdasarkan rentang tanggal
tanggal_awal = '2024-01-01'
tanggal_akhir = '2024-03-31'
data_filter = data[(data['date'] >= tanggal_awal) & (data['date'] <= tanggal_akhir)]

# Mengelompokkan data berdasarkan customer dan menghitung total pembelian
data_grouped = data_filter.groupby('customer')['quantity'].sum().sort_values(ascending=False).reset_index()

# Membuat grafik batang
plt.figure(figsize=(12, 6))
bars = plt.bar(data_grouped['customer'], data_grouped['quantity'])
plt.xlabel("Customer")
plt.ylabel("Total Pembelian")
plt.title(f"Analisis Pembelian Customer ({tanggal_awal} sampai {tanggal_akhir})")

# Menambahkan label jumlah pada setiap batang
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Menampilkan informasi barang yang paling banyak dan paling jarang dibeli
barang_terlaris = data_filter.groupby('product')['quantity'].sum().sort_values(ascending=False)
print("Barang yang paling banyak dibeli:")
print(barang_terlaris.head(1))
print("\nBarang yang paling jarang dibeli:")
print(barang_terlaris.tail(1))